In [ ]:
import os
import sys

import h5py
import numpy as np
import seaborn as sns

sys.path.insert(0, os.path.abspath("../.."))

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.collections import LineCollection

from transformers.data_utils.bb_data_loading import load_list
from transformers.training.jax_utils import cross_ent_loss
from transformers.training.utils import load_pickle

In [ ]:
t0012_list = load_list("../t0012/preference_data_1/day_list.txt")

In [ ]:
with h5py.File("../t0012/preference_data_1/t0012.hdf5") as f:
    sts_2 = f["states_2"][:]
    acts_2 = f["actions_2"][:]
    ts_2 = f["timesteps_2"][:]
    am_2 = f["attn_mask_2"][:]

    sts = f["states"][:]
    acts = f["actions"][:]
    ts = f["timesteps"][:]
    am = f["attn_mask"][:]

In [ ]:
pr = []
n_model = load_pickle("../t0012/results/pt_rewards/best_model.pkl")["model"]
for i in range(len(t0012_list)):
    sts_2[:, :, 15] = np.where(am_2 != 0, i, sts_2[:, :, 15])
    sts[:, :, 15] = np.where(am != 0, i, sts[:, :, 15])
    n_pred_2, n_weights_2 = n_model._train_state.apply_fn(
        n_model._train_state.params, sts_2, acts_2, ts_2, training=False, attn_mask=am_2
    )
    n_pred, n_weights = n_model._train_state.apply_fn(
        n_model._train_state.params, sts, acts, ts, training=False, attn_mask=am
    )
    B, T, _ = sts.shape
    trans_pred_1 = n_pred["weighted_sum"]
    trans_pred_2 = n_pred_2["weighted_sum"]

    sum_pred_1 = np.nanmean(trans_pred_1.reshape(B, T), axis=1).reshape(-1, 1)
    sum_pred_2 = np.nanmean(trans_pred_2.reshape(B, T), axis=1).reshape(-1, 1)

    pr.append(np.mean(1 / (np.exp(sum_pred_1 - sum_pred_2) + 1)))

In [ ]:
I_day = (np.max(pr) - np.min(pr)) / 4
print(I_day)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pr)
ax.set_title("Partial Dependence Plot for Day in Experiment")
ax.set_xlabel("Day")
ax.set_ylabel("Average Probability of Preferred Labeling for Segment 2")
plt.show()